# Processing Pipelines

In questa parte andremo a vedere cosa succede sotto il cofano quando si utilizza una pipeline SpaCy.

<br>

![pipeline](https://course.spacy.io/pipeline.png)

<br>

Una pipeline è una serie di funzioni da applicare ad un doc per aggiungere attributi come POS-tag, dipendenze, ...

Come si vede nell'immagine al momento la pipeline basica (SpaCy ha molte componenti da utilizzare in una pipeline) ha un testo in input poi in sequenza applica:
* tokenizer: suddivide il testo in token
* tagger: il POS tagging imposta i `token.tag` e `token.pos` (es. *"Apple is based in California"*, *Apple*: **POS**=PROPN e **TAG**=NNP) la differenza è che `.pos` e `.pos_` si riferiscono all'**universal POS tags** mentre `.tag` e `.tag_` al **fine-gradied POS tags**.
* parser: aggiunge gli attributi `token.head` e `token.dep` individua i token base nella frase e le dipendenze
* NER: indica le entità nel testo e aggiunge l'attributo `.ents` (come entità definiamo ad esempio COMPANY, MONEY, ... o altre regole che possiamo aggiungere manualmente con il *matching*)

e restituisce un oggetto `Doc`.

Tutte le pipeline che possiamo caricare in SpaCy hanno diversi file/componenti e un file di configurazione *config.cfg*, il config definisce le componenti base della pipeline e la lingua. Anche componenti predittive come ad esempio *tok2vec* preaddestrato (che serve per creare l'embedding del token) sono caricate nella pipeline.

<br>

![pipeline_comp](https://course.spacy.io/package_meta.png)

<br>

> altra componente della pipeline molto usata è il category labels, aggiunge l'attributo `.cats`, questo non si applica al token ma all'intero testo

Per vedere le componenti della pipeline bisogna usare `nlp.pipe_names`.

In [2]:
import spacy

nlp = spacy.load("en_core_web_sm")

print(nlp.pipe_names)

# ritorna la lista [nome funzione, components]
# print(nlp.pipeline)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


## Custom pipeline components

Le componenti custom permettono di inserire nuovi step personalizzati nella pipeline.

A seguito della tokenizzazione la pipeline SpaCy esegue le componenti in sequenza, a queste possono essere aggiunte quelle create da noi che verranno eseguite autometicamente quando si richiama la pipeline. Questi elementi sono molto utili per espandere i metadati del documento o per aggiungere attributi nella NER.

La pipeline è una funzione (o un *callable*) che prende un testo in input lo modifica e ce lo ritorna per essere processato al prossimo step.

Per aggiungere degli step bisogna usare il decorator `@Language.component` prima della funzione per creare lo step custom. A seguito della funzione bisogna aggiungere lo step alla pipeline con `nlp.add_pipe`.

Si può anche specificare in che posizione della pipeline deve essere inserito questo componente.

| Argomento | Descrizione | Esempio |
|:---:|:---:|:---|
| last | Se `True` aggiungi alla fine | `nlp.add_pipe("component", last=True)` |
| first | Se `True` aggiungi all0inizio | `nlp.add_pipe("component", first=True)` |
| before | Aggiunge prima del passo indicato | `nlp.add_pipe("component", before="ner")` |
| after | Aggiunge di seguito al passo indicato | `nlp.add_pipe("component", after="tagging")` |

Quando si crea la funzione per la componente custom è fondamentale non dimenticare di ritornare il `Doc` in output, così può essere l'input dello step successivo. Di seguito un esempio di un componente aggiuntivo che inserito all'inizio della pipeline ritorna la lunghezza del documento.

In [2]:
from spacy.language import Language

# Create the nlp object
nlp = spacy.load("en_core_web_sm")

# Define a custom component
@Language.component("custom_component")
def custom_component_function(doc):
    # Print the doc's length
    print("Doc length:", len(doc))
    # Return the doc object
    return doc

# Add the component first in the pipeline
nlp.add_pipe("custom_component", first=True)

# Print the pipeline component names
print("Pipeline:", nlp.pipe_names)

Pipeline: ['custom_component', 'tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [3]:
doc = nlp("Hello world! This is an example")

for token in doc:
    print(token.text)

Doc length: 7
Hello
world
!
This
is
an
example


Di seguito un esempio più complesso dove viene aggiunto uno step della pipeline dopo la NER per esolare solo alcune parole etichettate come *"ANIMAL"*.

In [4]:
import spacy
from spacy.language import Language
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

nlp = spacy.load("en_core_web_sm")
animals = ["Golden Retriever", "cat", "turtle", "Rattus norvegicus"]
animal_patterns = list(nlp.pipe(animals))
print("animal_patterns:", animal_patterns)
matcher = PhraseMatcher(nlp.vocab)
matcher.add("ANIMAL", animal_patterns)

# Define the custom component
@Language.component("animal_component")
def animal_component_function(doc):
    # Apply the matcher to the doc
    matches = matcher(doc)
    # Create a Span for each match and assign the label "ANIMAL"
    spans = [Span(doc, start, end, label="ANIMAL") for match_id, start, end in matches]
    # Overwrite the doc.ents with the matched spans
    doc.ents = spans
    return doc


# Add the component to the pipeline after the "ner" component
nlp.add_pipe("animal_component", after="ner")
print(nlp.pipe_names)

# Process the text and print the text and label for the doc.ents
doc = nlp("I have a cat and a Golden Retriever")
print([(ent.text, ent.label_) for ent in doc.ents])

animal_patterns: [Golden Retriever, cat, turtle, Rattus norvegicus]
['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner', 'animal_component']
[('cat', 'ANIMAL'), ('Golden Retriever', 'ANIMAL')]


## Aggiungere attributi custom

In questo paragrafo vedremo come estendere gli oggetti `Doc`, `Token` e `Span`.

Con gli attributi custom si possono estendere gli oggetti a piacimento, l'aggiunta può essere manuale o dinamica e automatizzata. Per accedere agli attributi si usa `._`, questo fa capire che l'attributo è stato aggiunto dall'utente e non dalla pipeline di spacy.

Per registrarli globalmente bisogna usare la funzione `set_extension` di `spacy.tokens`

In [5]:
# Import global classes
from spacy.tokens import Doc, Token, Span

# Set extensions on the Doc, Token and Span
Doc.set_extension("title", default=None)
Token.set_extension("_color", default=False)


Il primo elemento è il nome dell'attributo, il secondo il valore di default. Se esiste, come in questo caso, può essere sovrascritto.

In [6]:
doc._.title = "My document"
token._._color = False

esistono tre tipi di estensioni:
* Attribute extensions
* Property extensions
* Method extensions

Attribute extension imposta un valore che può essere sovrascritto, ad esempio vogliamo sapere se una parola in una frase è un colore, impostiamo un nuovo attributo `is_color` a `False` e se la parola è un colore verrà sovrascritto con `True`.


In [7]:
doc = nlp("The sky is blue.")

# Overwrite extension attribute value
doc[3]._._color = True

Le Property exension lavorando come le classiche property in python. Hanno una metodo *get* che aiuta a cercare il valore dell'attributo impostato. Nel getter dobbiamo impostare un solo argomento, l'oggetto su cui applicarlo in questo caso il token. Il nostro metodo ci dirà se è un colore oppure no.

> caso particolare sono le span extension. Funzionano come le Property ma si applicano agli span

In [8]:
from spacy.tokens import Token

# Define getter function
def get_is_color(token):
    colors = ["red", "yellow", "blue"]
    return token.text in colors

# Set extension on the Token with getter
Token.set_extension("is_color", getter=get_is_color)

doc = nlp("The sky is blue.")
print(doc[3]._.is_color, "-", doc[3].text)

True - blue


In [ ]:
"""
# Esempio con Span Extension

import spacy
from spacy.tokens import Span

nlp = spacy.load("en_core_web_sm")


def get_wikipedia_url(span):
    # Get a Wikipedia URL if the span has one of the labels
    if span.label_ in ("PERSON", "ORG", "GPE", "LOCATION"):
        entity_text = span.text.replace(" ", "_")
        return "https://en.wikipedia.org/w/index.php?search=" + entity_text


# Set the Span extension wikipedia_url using the getter get_wikipedia_url
Span.set_extension("wikipedia_url", getter=get_wikipedia_url)

doc = nlp(
    "In over fifty years from his very first recordings right through to his "
    "last album, David Bowie was at the vanguard of contemporary culture."
)
for ent in doc.ents:
    # Print the text and Wikipedia URL of the entity
    print(ent.text, ent._.wikipedia_url)
"""

Le Method extension creano un metodo richiamabile passando degli argomenti. Nel nostro caso creiamo un metodo `has_color` che data una stringa ci dice se è un colore oppure no.

In [9]:
from spacy.tokens import Doc

# Define method with arguments
def has_token(doc, token_text):
    in_doc = token_text in [token.text for token in doc]
    return in_doc

# Set extension on the Doc with method
Doc.set_extension("has_token", method=has_token)

doc = nlp("The sky is blue.")
print(doc._.has_token("blue"), "- blue")
print(doc._.has_token("cloud"), "- cloud")

True - blue
False - cloud


## Scalare le performance

Quando bisogna processare molto testo è bene usare `nlp.pipe` al posto del classico `nlp`. `nlp.pipe` prende `Doc` e lo processa in stream a differenza di `nlp` che deve essere applicato su ogni testo. `nlp.pipe` restituisce un *iterator* quindi se si vuole una lista di `Doc` meglio richiamarlo tra [].

```
docs = list(nlp.pipe(LOTS_OF_TEXTS))
```

Se si imposta il parametro `as_tuples=True` possono essere passate a `nlp.pipe` tuple di testo\contesto, questo è utile per passare metadati.

In [3]:
data = [
    ("This is a text", {"id": 1, "page_number": 15}),
    ("And another text", {"id": 2, "page_number": 16}),
]

for doc, context in nlp.pipe(data, as_tuples=True):
    print(doc.text, context["page_number"])

This is a text 15
And another text 16


Con `nlp.pipe` e le extensions si possono aggiungere i metadati agli attributi. In questo esempio vengono impostati due attributi *None* e vengono popolati con i metadati e `nlp.pipe`.

In [ ]:
from spacy.tokens import Doc

Doc.set_extension("id", default=None)
Doc.set_extension("page_number", default=None)

data = [
    ("This is a text", {"id": 1, "page_number": 15}),
    ("And another text", {"id": 2, "page_number": 16}),
]

for doc, context in nlp.pipe(data, as_tuples=True):
    doc._.id = context["id"]
    doc._.page_number = context["page_number"]

### Adattare la pipeline

Uno scenario comune è la personalizzazione della pipeline. Abbiamo visto in precedenza come aggiungere step ma può capitare di avere step che non ci sono utili e che prendono del tempo e risorse. Possiamo eliminare o isolare questi elementi con un semplice comando.

* `nlp.make_doc`: prende un testo e lo trasforma in doc applicando il tokenizer
* `nlp.select_pipe(disable=[...])`: disabilita alcune componenti della pipeline temporaneamente

In [24]:
# Disable tagger and parser
with nlp.select_pipes(disable=["tagger", "parser"]):
    # Process the text and print the entities
    doc = nlp("Hello World!")
    print(doc.ents)

()


/home/nico/anaconda3/envs/nlp/lib/python3.8/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
